In [1]:
import pandas as pd
import ast
import logging
from typing import Any, List, Tuple

In [2]:
# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s — %(levelname)s — %(message)s")
logger = logging.getLogger(__name__)


def load_and_clean_csv(file_path: str) -> pd.DataFrame:
    """Load a CSV file and clean column names."""
    try:
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.strip().str.lower()
        logger.info(f"Loaded and cleaned: {file_path} — {df.shape[0]} rows.")
        return df
    except Exception as e:
        logger.error(f"Failed to load {file_path}: {e}")
        raise


In [3]:
def safe_eval(val: Any) -> Any:
    """Safely convert a string to a Python list if it looks like one."""
    try:
        return ast.literal_eval(val) if isinstance(val, str) and val.startswith("[") else val
    except (ValueError, SyntaxError) as e:
        logger.warning(f"safe_eval failed for value: {val} — {e}")
        return val

In [4]:
def apply_safe_eval(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    """Apply safe_eval to specified columns."""
    for col in columns:
        df[col] = df[col].apply(safe_eval)
        logger.info(f"Applied safe_eval on column: {col}")
    return df

In [5]:
def merge_recommendations_with_schemes(
    df_recs: pd.DataFrame,
    df_schemes: pd.DataFrame
) -> pd.DataFrame:
    """Combine recommended products with available schemes."""
    results = []

    for _, row in df_recs.iterrows():
        partner_id = row.get("partner_id")
        recommended_products = row.get("recommended_products", [])
        similarity_scores = row.get("similarity_scores", [])

        if not isinstance(recommended_products, list) or not isinstance(similarity_scores, list):
            logger.warning(f"Skipping invalid row for Partner {partner_id}")
            continue

        for product, score in zip(recommended_products, similarity_scores):
            matched_schemes = df_schemes[df_schemes["product_id"] == product][["scheme_1", "scheme_2", "scheme_3"]]
            
            if not matched_schemes.empty:
                scheme_1, scheme_2, scheme_3 = matched_schemes.iloc[0].fillna("Not Available").values
            else:
                scheme_1, scheme_2, scheme_3 = "Not Available", "Not Available", "Not Available"
            
            results.append([partner_id, product, score, scheme_1, scheme_2, scheme_3])

    return pd.DataFrame(results, columns=[
        "Partner_id", "Product_id", "Similarity_Scores", "Scheme_1", "Scheme_2", "Scheme_3"
    ])


In [6]:
def generate_final_scheme_output(
    recs_path: str,
    scheme_path: str,
    output_path: str
):
    """End-to-end function to generate final scheme mapping from recommendations."""
    df_recs = load_and_clean_csv(recs_path)
    df_schemes = load_and_clean_csv(scheme_path)

    df_recs = apply_safe_eval(df_recs, ["recommended_products", "similarity_scores"])
    df_schemes["partner_id"] = df_schemes["partner_id"].apply(safe_eval)

    final_df = merge_recommendations_with_schemes(df_recs, df_schemes)
    final_df.to_csv(output_path, index=False)
    logger.info(f"Final partner-product scheme mapping saved to {output_path}")


if __name__ == "__main__":
    generate_final_scheme_output(
        recs_path="User_Based_Recommendations.csv",
        scheme_path="Top_Optimized_Schemes_with_LP.csv",
        output_path="Final_Partner_Product_Schemes.csv"
    )

2025-04-02 09:30:48,465 — INFO — Loaded and cleaned: User_Based_Recommendations.csv — 1000 rows.
2025-04-02 09:30:48,469 — INFO — Loaded and cleaned: Top_Optimized_Schemes_with_LP.csv — 15 rows.
2025-04-02 09:30:48,484 — INFO — Applied safe_eval on column: recommended_products
2025-04-02 09:30:48,496 — INFO — Applied safe_eval on column: similarity_scores
2025-04-02 09:30:50,951 — INFO — Final partner-product scheme mapping saved to Final_Partner_Product_Schemes.csv
